## **Análisis del COVID-19 en México**

Se analizarán los datos confirmados, negativos y las defunciones a nivel municipal y nacional, incluso se comparará con datos de casos positivos y defunciones en Estados Unidos.

Las bases de datos utilizadas se encuentran en este link: https://drive.google.com/drive/folders/1MR1dJRcQxDRAr--aRmHhrR3FR0XG75fs?usp=sharing
C_M.csv <- Casos confirmados por municipio
C_N.csv <- Casos confirmados por estado
N_M.csv <- Casos negativos por municipio
N_N.csv <- Casos negativos por estado
D_M.csv <- Defunciones por municipio
D_N.csv <- Defunciones por estado
national-history.csv <- Datos de Estados Unidos
mapa_casos_geo.html <- Mapa Geográfico Interactivo

1. Antes de realizar un análisis más pronfundo, visualizaremos a través de un mapa geográfico interactivo, el porcentaje de datos confirmados desde que comenzó la pandemia 26/02/20 hasta el día 13/05/23 a nivel municipal en México.

In [2]:
# Cargamos las librerías a utilizar.
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import folium

# Habilitamos google drive.
from google.colab import drive
drive.mount('/content/drive')

# Cargamos el marco geoestadístico.
mx = gpd.read_file('/content/drive/MyDrive/mapa_mexico/') \
        .set_index('CLAVE') \
        .to_crs(epsg=4485)

# Base de datos de casos confirmados.
confirmados = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_M.csv")

# Calcular los porcentajes por municipio.
confirmados["total"] = confirmados.iloc[:, 3:].sum(axis=1)
confirmados["percent"] = confirmados["total"] / confirmados["poblacion"]

# Fusionar los datos geográficos con los datos confirmados.
mx.index = mx.index.astype("int")
mx_covid = mx.merge(confirmados, left_index=True, right_on="cve_ent")

# Crear un mapa interactivo utilizando folium.
m = folium.Map(location=[24, -102], zoom_start=6, tiles='Stamen Terrain')

# Agregar la capa de geometría del marco geoestadístico.
folium.GeoJson(mx_covid).add_to(m)

# Agregar el mapa de calor de los porcentajes de casos confirmados.
folium.Choropleth(
    geo_data=mx_covid,
    data=mx_covid,
    columns=['cve_ent', 'percent'],
    key_on='feature.properties.cve_ent',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% de casos positivos de Covid-19 por municipio'
).add_to(m)

# Mostrar el mapa interactivo.
m

# Guardamos el mapa interactivo en un archivo html.
m.save('mapa_casos_geo.html')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.3 MB/s eta 0:00:00
Mounted at /content/drive


¡Es importante tomar en cuenta que el archivo obtenido no se puede visualizar aquí, por lo que se encuentra adjunto y en caso de correr el código debe descargarse dicho archivo html!

Gracias a la paleta de colores utilizada es posible observar primero que nada, la zona donde se concentraron los valores positivos que es donde más color rojo y naranja vemos, que son Baja California Sur y Ciudad de México.
La Ciudad de México y parte del Estado de México parece ser la zona que más casos positivos tuvo y es razonable dado que la Ciudad de México es el núcleo urbano más grande del país.


2. El gráfico geográfico puede no ser para todos, pues es un archivo pesado e incluso pudiera ser un poco lento de mover, ¿Habrá algún gráfico que represente la información resumida?

In [2]:
import plotly.express as px

# Cargamos los datos desde Google Drive
df = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv")

# Excluir la última fila del DataFrame
df = df.iloc[:-1]

df_selec = df.melt(id_vars=["cve_ent", "poblacion", "nombre"], var_name="Fecha", value_name="Casos")


# Crear el gráfico utilizando el nombre de la entidad como etiquetas
fig = px.line(df_selec, x="Fecha", y="Casos", color="nombre", title="Casos positivos de COVID-19 por entidad federativa.")
fig.update_layout(xaxis_title="", yaxis_title="Casos positivos", xaxis=dict(showticklabels=False))
fig.show()


El gráfico de lineas de la librería Plotly nos ayuda a observar la distribución de los casos positivos de Covid-19 por entidad federativa, así a simple vista es muy claro que la entidad con más casos siempre fue el Distrito Federal (Ciudad de México), pues igual es la entidad con más población. Jugando con la gráfica es posible observar que las entidades que parecieran ser las que tuvieron menos casos positivos fueron Campeche, Tlaxcala y Chiapas.

3. Si no se quisiera información tan detallada, ¿Es posible obtener solo los casos totales a nivel nacional? 

In [3]:
# Obtener la última fila que contiene los datos a nivel nacional.
ultima_fila = df.iloc[-1]

fechas = ultima_fila[3:].index
casos_positivos = ultima_fila[3:].values

# Creamos un DataFrame.
df_animacion = pd.DataFrame({"Fecha": fechas, "Casos": casos_positivos})

# Gráfica animada utilizando Plotly.
fig = px.bar(df_animacion, x="Fecha", y="Casos", title="Casos positivos de COVID-19 en México.",
             labels={"Fecha": "Fecha", "Casos": "Casos positivos"},
             template="plotly")
fig.update_layout(barmode="stack")
fig.update_xaxes(type='category')

# Configuramos la animación.
fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, {"frame": {"duration": 30, "redraw": True},
                                                         "fromcurrent": True, "transition": {"duration": 0}}])])])

fig.update_layout(annotations=[dict(text=' ', showarrow=False, x=0.5, y=-0.1, xref='paper', yref='paper')],
                  height=500)
frames = [dict(data=[dict(type='bar', x=df_animacion["Fecha"][:i+1], y=df_animacion["Casos"][:i+1])]) for i in range(len(df_animacion))]
fig.update(frames=frames)

# Mostrar la gráfica animada.
fig.show()


Output hidden; open in https://colab.research.google.com to view.

Se pueden seleccionar solo los datos totales para ver su evolución con el paso del tiempo, la gráfica interactiva y con animación obtenida a partir de la libería Plotly nos permite observar el comportamiento de los casos positivos a lo largo de los días desde que comenzó la pendemia, por eso mismo dejamos la fecha en el eje X, al igual que en la anterior gráfica se puede observar claramente las 4 olas de covid y que la tercera fue la peor a nivel nacional.

4. Habrá quienes al ver la cantidad de casos positivos se pregunten sobre lo que nos dejó la pandemia, en específico, ¿Cuántas muertes nos dejó?

In [4]:
# Eliminar la última fila de datos.
df2 = df.iloc[:-1]

# Seleccionamos las columnas que contienen los datos de defunciones.
columnas_defunciones = df2.columns[3:]

# Calcular el total de defunciones por estado.
df2["total_defunciones"] = df2[columnas_defunciones].sum(axis=1)

# Crear el treemap utilizando Plotly.
fig = px.treemap(df2, path=['nombre'], values='total_defunciones',
                 title='Defunciones por COVID-19 en México',
                 color_continuous_scale='plasma')

# Mostrar el treemap.
fig.show()


<ipython-input-4-c4b4c9438604>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Con el TreeMap de la librería Plotly, es posible obtener el total de casos por estado durante toda la pandemia, desgraciadamente podemos ver que tanto la Ciudad de México (Distrito Federal) y Estado de México (México), tuvieron más de 80,000 defunciones cada uno, lo cual es lamentable.

5. Después de ver el TreeMap pudieramos cuestionarnos si en realidad existe relación entre la cantidad de defunciones con la cantidad de casos positivos.

In [5]:
import pandas as pd
import plotly.graph_objects as go

# Cargamos los datos de defunciones y casos positivos.
df_defunciones = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/D_N.csv")
df_casos_positivos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv")

# Omitimos las últimas filas.
df_defunciones = df_defunciones[:-1]
df_casos_positivos = df_casos_positivos[:-1]

# Obtenemos las fechas.
columnas_fechas = df_defunciones.columns[3:]

# Total de defunciones y casos positivos por mes.
df_defunciones["Total_defunciones"] = df_defunciones[columnas_fechas].sum(axis=1)
df_casos_positivos["Total_casos_positivos"] = df_casos_positivos[columnas_fechas].sum(axis=1)

# Combinamos los datos en un solo DataFrame.
df_combined = pd.merge(df_defunciones[["nombre", "Total_defunciones"]], df_casos_positivos[["nombre", "Total_casos_positivos"]], on="nombre")

# Creamos el gráfico interactivo.
fig = go.Figure()
fig.add_trace(go.Bar(x=df_combined["nombre"], y=df_combined["Total_defunciones"], name="Defunciones", marker_color="deep sky blue"))
fig.add_trace(go.Bar(x=df_combined["nombre"], y=df_combined["Total_casos_positivos"], name="Casos Positivos", marker_color="salmon"))

# Configuramos el diseño.
fig.update_layout(title="Comparación de Defunciones y Casos Positivos por Estado",
                  xaxis_title="Estado",
                  yaxis_title="",
                  barmode="group")

# Mostrar el gráfico interactivo.
fig.show()


<ipython-input-5-a6e0ee715dc7>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Al hacerle zoom o directamente de la gráfica sin interacción es posible notar que los estados en los que se reportaron mayores casos positivos fueron los que reportaron mayor cantidad de defunciones, así que nada más con esto se concluye que es muy probable que sí exista relación.

6. Así como se hizo la comparación de los casos positivos con las defunciones, ¿Es posible analizar los casos negativos con los casos positivos por Covid-19?

In [6]:
# Cargar los datos de casos negativos y positivos.
df_negativos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/N_N.csv").iloc[[-1]]
df_positivos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv").iloc[[-1]]

# Obtenemos las fechas y cantidades de casos negativos y positivos.
fechas = df_negativos.columns[3:]
casos_negativos = df_negativos.iloc[:, 3:].values.flatten()
casos_positivos = df_positivos.iloc[:, 3:].values.flatten()


# Creamos el gráfico interactivo.
fig = go.Figure()
fig.add_trace(go.Scatter(x=fechas, y=casos_negativos, name='Casos Negativos',
                         mode='markers', marker=dict(color='deep sky blue', size=8),
                         hovertemplate='<b>Casos Negativos</b><br>'
                                       'Fecha: %{x}<br>'
                                       'Cantidad de Casos: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=fechas, y=casos_positivos, name='Casos Positivos',
                         mode='markers', marker=dict(color='salmon', size=8),
                         hovertemplate='<b>Casos Positivos</b><br>'
                                       'Fecha: %{x}<br>'
                                       'Cantidad de Casos: %{y}<extra></extra>'))

# Configuraramos el diseño del gráfico.
fig.update_layout(title='Casos Positivos y Negativos de COVID-19 en México.',
                  xaxis_title=' ',
                  yaxis_title='Cantidad de Casos',
                  legend_title='Tipo de Caso',xaxis=dict(showticklabels=False),
                  showlegend=True)

# Mostramos el gráfico interactivo.
fig.show()


Es muy fácil observar que al principio hubieron más casos positivos que negativos y esto pudo deberse a que solo te hacían pruebas si tenías síntomas, sin embargo, conforme pasó el tiempo las pruebas comenzaron a estar al alcance de todos y quizá es debido a esto que después erán más los casos negativos que positivos.

7. Incluso podríamos observar el comportamiento del covid dado algún estado específico, podríamos tomar de referencia a la capital del país, la Ciudad de México.

In [7]:
# Cargamos los datos positivos y negativos.
casos_positivos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv")
casos_negativos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/N_N.csv")

# Filtramos los datos solo para el Distrito Federal.
df_positivos = casos_positivos[casos_positivos["nombre"] == "DISTRITO FEDERAL"]
df_negativos = casos_negativos[casos_negativos["nombre"] == "DISTRITO FEDERAL"]

# Melt para transformar datos a formato largo.
df_positivos_largo = pd.melt(df_positivos, id_vars=["cve_ent", "nombre", "poblacion"],
                             var_name="fecha", value_name="casos_positivos")
df_negativos_largo = pd.melt(df_negativos, id_vars=["cve_ent", "nombre", "poblacion"],
                             var_name="fecha", value_name="casos_negativos")


# Combinar datos de casos positivos y negativos.
df_combinado = pd.merge(df_positivos_largo, df_negativos_largo, on=["cve_ent", "nombre", "poblacion", "fecha"])

# Creamos el gráfico interactivo.
fig = px.line(df_combinado, x="fecha", y=["casos_positivos", "casos_negativos"],
              labels={"value": "Casos", "variable": "Tipo de Casos"},
              title="Evolución de casos positivos y negativos de COVID-19 en el Distrito Federal")
fig.update_layout(xaxis_title=' ',
                  yaxis_title='Cantidad de Casos',
                  legend_title='Tipo de Casos',xaxis=dict(showticklabels=False),
                  showlegend=True)

# Mostramos gráfico interactivo.
fig.show()

Ahora teniendo solo los datos al paso del tiempo para el Distrito Federal, podemos ver que los datos negativos (ahora en color rojo) superaron casi siempre a los casos positivos, incluso parece que en la tercera ola, lo que luce diferente en los casos a nivel nacional, por eso resulta importante hacer un análisis a nivel entidad federativa.


8. Observamos los datos por el mismo estado pero, ¿Se podría comparar el estado con mayores casos positivos con el de menores casos positivos?

In [8]:
# Total de casos positivos y negativos por entidad.
total_positivos = casos_positivos.iloc[:, 3:].sum(axis=1)
total_negativos = casos_negativos.iloc[:, 3:].sum(axis=1)

indice_max_positivos = total_positivos.idxmax()
indice_min_positivos = total_positivos.idxmin()

# Entidad con el mayor número de casos positivos.
entidad_max_positivos = casos_positivos.loc[indice_max_positivos, "nombre"]

# Entidad con el menor número de casos positivos.
entidad_min_positivos = casos_positivos.loc[indice_min_positivos, "nombre"]

# Mostrar los resultados.
print("Entidad con mayor número de casos positivos:", entidad_max_positivos)
print("Entidad con menor número de casos positivos:", entidad_min_positivos)

Entidad con mayor número de casos positivos: Nacional
Entidad con menor número de casos positivos: CAMPECHE


In [9]:
# Cargamos datos de casos positivos.
casos_positivos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv")

# Filtramos datos para Ciudad de México y Campeche.
ciudad_mexico = casos_positivos[casos_positivos["nombre"] == "DISTRITO FEDERAL"]
campeche = casos_positivos[casos_positivos["nombre"] == "CAMPECHE"]

# Melt para transformar datos a formato largo.
ciudad_mexico_largo = pd.melt(ciudad_mexico, id_vars=["cve_ent", "nombre", "poblacion"], 
                              var_name="fecha", value_name="casos_positivos")
campeche_largo = pd.melt(campeche, id_vars=["cve_ent", "nombre", "poblacion"], 
                         var_name="fecha", value_name="casos_positivos")
ciudad_mexico_largo["fecha"] = pd.to_datetime(ciudad_mexico_largo["fecha"], format="%d-%m-%Y")
campeche_largo["fecha"] = pd.to_datetime(campeche_largo["fecha"], format="%d-%m-%Y")

# Rango de fechas deseado.
fecha_inicio = pd.to_datetime("05-12-2021", format="%d-%m-%Y")
fecha_fin = pd.to_datetime("19-04-2022", format="%d-%m-%Y")
ciudad_mexico_largo = ciudad_mexico_largo[(ciudad_mexico_largo["fecha"] >= fecha_inicio) & (ciudad_mexico_largo["fecha"] <= fecha_fin)]
campeche_largo = campeche_largo[(campeche_largo["fecha"] >= fecha_inicio) & (campeche_largo["fecha"] <= fecha_fin)]

# Concatenar los datos de Ciudad de México y Campeche
datos_comparacion = pd.concat([ciudad_mexico_largo, campeche_largo])

# Creamos gráfico de área interactivo.
fig = px.area(datos_comparacion, x="fecha", y="casos_positivos", color="nombre", title="Comparación de casos positivos de COVID-19: Ciudad de México vs. Campeche")
fig.update_xaxes(rangeslider_visible=True)  # Agregar control deslizante para zoom en el eje x

# Mostramos gráfico interactivo.
fig.show()

Para este caso decidí solo mostrar los datos referentes a la "tercera ola" que va desde finales de diciembre de 2021 a inicios de marzo de 2022, podemos ver que incluso en el momento en que el Covid-19 estuvo más fuerte, el estado de Campeche no aumentó drásticamente sus casos como la Ciudad de México.

9. Ya realizamos el análisis de casos positivos, casos negativos, incluso de las defunciones en general pero valdría la pena ver el progreso de estos datos.

In [10]:
#Datos de defunciones
df = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/D_N.csv")
df = df.iloc[:-1]

df_selec = df.melt(id_vars=["cve_ent", "poblacion", "nombre"], var_name="Fecha", value_name="Casos")

In [11]:
fig = px.scatter(df_selec, x="Fecha", y="Casos", color="Casos", title="Casos de defunciones por COVID-19 en México.",
                 color_continuous_scale="Viridis")
fig.update_layout(xaxis_title="", yaxis_title="Casos positivos", xaxis=dict(showticklabels=False))
fig.show()

Es impresionante ver como las defunciones fueron decreciendo conforme el paso de la pandemia, sabemos que la vacunación de COVID-19 inició en diciembre del 2020, pero en lo que la vacuna llegó a todo México transcurrieron un par de meses, quizá es por eso que las defunciones disminuyeron drásticamente a inicios de Abril de 2021.

10. Y después de ver la situación en México ¿Cómo estuvo la situación en Estados Unidos?
La base de datos fue obtenida de: https://covidtracking.com/data/download


In [12]:
import pandas as pd
import plotly.express as px

# Cargar los datos desde el archivo CSV
data = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/national-history.csv")

# Crear el gráfico interactivo de área apilada
fig = px.area(data, x='date', y='positiveIncrease', title='Casos positivos de COVID-19 en Estados Unidos a través del tiempo',
              labels={'date': 'Fecha', 'positiveIncrease': 'Casos positivos'},
              template='plotly_white')

# Mostrar el gráfico interactivo
fig.show()


Este gráfico nos muestra los casos positivos de COVID-19 en todo Estados Unidos en el periodo del 26 de febrero de 2020 al 7 de marzo de 2021, podemos observar como aumentan drásticamente los datos.

11. Para hacer una comparación más justa, ¿será posible observar los datos de la misma fecha para Estados Unidos y México?

In [13]:
# Cargamos datos de casos positivos.
casos_positivos = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/C_N.csv")

# Filtrar datos para México.
nacional = casos_positivos[casos_positivos["nombre"] == "Nacional"]

nacional_largo = pd.melt(nacional, id_vars=["cve_ent", "nombre", "poblacion"], 
                         var_name="fecha", value_name="casos_positivos")

nacional_largo["fecha"] = pd.to_datetime(nacional_largo["fecha"], format="%d-%m-%Y")

# Filtramos los datos por el rango de fechas deseado.
fecha_inicio = pd.to_datetime("26-02-2020", format="%d-%m-%Y")
fecha_fin = pd.to_datetime("07-03-2021", format="%d-%m-%Y")
nacional_largo = nacional_largo[(nacional_largo["fecha"] >= fecha_inicio) & (nacional_largo["fecha"] <= fecha_fin)]

# Cargar los datos de Estados Unidos
data = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/national-history.csv")
data = data.iloc[:-44]
fig = go.Figure()

fig.add_trace(go.Scatter(x=nacional_largo["fecha"], y=nacional_largo["casos_positivos"], 
                         name="Casos positivos (Nacional)", mode="lines"))
fig.add_trace(go.Scatter(x=data["date"], y=data["positiveIncrease"], 
                         name="Casos positivos (Estados Unidos)", mode="lines"))

# Configurar el diseño del gráfico
fig.update_layout(title="Comparación de casos positivos de COVID-19: Nacional vs. Estados Unidos",
                  xaxis_title="Fecha", yaxis_title="Casos positivos",
                  template="plotly_white")

# Mostrar el gráfico interactivo
fig.show()


Dado el gráfico podría resultar dificil hacer una comparación sin tener que hacerle zoom al gráfico y es que al parecer los datos positivos de Estados Unidos parecieran multiplicarse por 10 con respecto a México cuando solo tienen poco más del doble de población, quizá se pudo deber a que hacían más pruebas a su población.

12. Ahora quizá toca preguntarse con el caso de las defunciones, si Estados Unidos tuvo más casos positivos se podría pensar que igual tuvo más defunciones, ¿será cierto?

In [14]:
# Datos de defunciones.
casos_defunciones = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/D_N.csv")

# Filtramos para nivel nacional.
nacional = casos_defunciones[casos_defunciones["nombre"] == "Nacional"]

nacional_largo = pd.melt(nacional, id_vars=["cve_ent", "nombre", "poblacion"],
                         var_name="fecha", value_name="casos_defunciones")
nacional_largo["fecha"] = pd.to_datetime(nacional_largo["fecha"], format="%d-%m-%Y")

fecha_inicio = pd.to_datetime("26-02-2020", format="%d-%m-%Y")
fecha_fin = pd.to_datetime("07-03-2021", format="%d-%m-%Y")
nacional_largo = nacional_largo[(nacional_largo["fecha"] >= fecha_inicio) & (nacional_largo["fecha"] <= fecha_fin)]

# Cargar los datos desde el archivo CSV.
data = pd.read_csv("/content/drive/MyDrive/Proyecto_Final_VD/national-history.csv")
data = data.iloc[:-44]
fig = go.Figure()

# Primera barra de casos positivos.
fig.add_trace(go.Bar(x=nacional_largo["fecha"], y=nacional_largo["casos_defunciones"],
                     name="Casos de defunciones (México)"))

# Segunda barra de casos positivos.
fig.add_trace(go.Bar(x=data["date"], y=data["deathIncrease"],
                     name="Casos de defunciones (Estados Unidos)"))

# Configuramos el diseño del gráfico.
fig.update_layout(barmode="group",
                  title="Comparación de defunciones a causa de COVID-19: México vs. Estados Unidos",
                  xaxis_title="Fecha", yaxis_title="Defunciones",
                  template="plotly_white")

# Mostramos el gráfico interactivo
fig.show()


Resulta triste notar que a pesar de que en este periodo nunca superamos ni un poco a Estados Unidos en casos positivos, por las fechas de Julio 2020 tuvimos más defunciones, eso pudo ser resultado de que su atención a pacientes fue mejor y que su población se vacuno desde antes.